In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [5]:
training_data = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor()
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [10]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader:
    print(f'Shape of X [N, C, H, W] \t{X.shape}')
    print(f'Shape of y:\t\t\t{y.shape} {y.dtype}')
    break

Shape of X [N, C, H, W] 	torch.Size([64, 1, 28, 28])
Shape of y:			torch.Size([64]) torch.int64


To define a neural network in PyTorch, we create a class that inherits from nn.Module. We define the layers of the network in the `__init__` function and specify how data will pass through the network in the `forward` function. To accelerate operations in the neural network, we move it to the GPU if available.

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Using cpu device


In [13]:
if 'model' in locals():
    del model
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [15]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f}, [{current:>5d}/{size:>5d}]')

In [18]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    
    print(f'Test error: \nAccuracy: {(100*correct):>0.1f}%, Avg. Loss: {test_loss:>8f}\n')

In [19]:
epochs = 5
for t in range(epochs):
    print(f'Epoch {t+1}\n-------------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print('Done!')

Epoch 1
-------------------------------
loss: 2.131975, [    0/60000]
loss: 2.118197, [ 6400/60000]
loss: 2.051799, [12800/60000]
loss: 2.066291, [19200/60000]
loss: 2.019424, [25600/60000]
loss: 1.933532, [32000/60000]
loss: 1.957671, [38400/60000]
loss: 1.878325, [44800/60000]
loss: 1.884487, [51200/60000]
loss: 1.784619, [57600/60000]
Test error: 
Accuracy: 57.7%, Avg. Loss: 1.804405

Epoch 2
-------------------------------
loss: 1.842132, [    0/60000]
loss: 1.806780, [ 6400/60000]
loss: 1.685215, [12800/60000]
loss: 1.729423, [19200/60000]
loss: 1.631997, [25600/60000]
loss: 1.568117, [32000/60000]
loss: 1.583501, [38400/60000]
loss: 1.503213, [44800/60000]
loss: 1.530091, [51200/60000]
loss: 1.412169, [57600/60000]
Test error: 
Accuracy: 60.6%, Avg. Loss: 1.447159

Epoch 3
-------------------------------
loss: 1.515423, [    0/60000]
loss: 1.478342, [ 6400/60000]
loss: 1.335996, [12800/60000]
loss: 1.415940, [19200/60000]
loss: 1.306542, [25600/60000]
loss: 1.290659, [32000/60000

In [20]:
torch.save(model.state_dict(), 'model.pth')
print('Saved PyTorch model to model.pth')

Saved PyTorch model to model.pth
